<a href="https://colab.research.google.com/github/ash7-g/AI-1BM23CS400/blob/main/PrepositionLogic1BM23CS400.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re

class KnowledgeBase:
    def __init__(self):
        self.kb = []  # Knowledge base to store logical sentences

    def add(self, statement):
        """Add a statement (logical formula) to the knowledge base."""
        self.kb.append(statement)

    def is_in_kb(self, statement):
        """Check if a statement is in the knowledge base."""
        return statement in self.kb

    def infer(self):
        """Perform inference using Modus Ponens and other inference rules."""
        new_facts = set()  # To keep track of newly inferred facts
        for sentence in self.kb:
            if isinstance(sentence, Implication):  # If it's an implication (p → q)
                premise, conclusion = sentence.premise, sentence.conclusion
                if premise in self.kb:  # If the premise is known
                    new_facts.add(conclusion)  # We can infer the conclusion
            elif isinstance(sentence, Disjunction):  # If it's a disjunction (p ∨ q)
                left, right = sentence.left, sentence.right
                if left in self.kb:
                    new_facts.add(right)  # If left side is true, we can infer right side
                elif right in self.kb:
                    new_facts.add(left)  # If right side is true, we can infer left side
        return new_facts

    def display(self):
        """Display the current knowledge base."""
        for statement in self.kb:
            print(statement)

    def check_alpha(self, alpha):
        """Check if alpha can be inferred from the knowledge base."""
        # Perform inference until no more facts can be derived
        inferred = self.infer()
        while inferred:
            for fact in inferred:
                if fact == alpha:
                    return True  # Alpha has been inferred
            self.kb.extend(inferred)
            inferred = self.infer()
        return False


class Statement:
    """Base class for logical statements."""
    def __str__(self):
        pass


class Literal(Statement):
    """Represents a literal, a basic propositional variable (e.g., p, q)."""
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return self.name


class Negation(Statement):
    """Represents a negation (¬p)."""
    def __init__(self, statement):
        self.statement = statement

    def __str__(self):
        return f"¬({self.statement})"


class Implication(Statement):
    """Represents an implication (p → q)."""
    def __init__(self, premise, conclusion):
        self.premise = premise
        self.conclusion = conclusion

    def __str__(self):
        return f"({self.premise} → {self.conclusion})"


class Disjunction(Statement):
    """Represents a disjunction (p ∨ q)."""
    def __init__(self, left, right):
        self.left = left
        self.right = right

    def __str__(self):
        return f"({self.left} ∨ {self.right})"


def parse_formula(formula):
    """Parse a formula and return the appropriate logical statement object."""
    formula = formula.replace(" ", "")  # Remove spaces

    # Match literals (e.g., p, q, r)
    match_literal = re.match(r"^[a-zA-Z]+$", formula)
    if match_literal:
        return Literal(formula)

    # Match negation (e.g., ¬p, NOT p, ~p) - handling nested negations
    match_negation = re.match(r"^(NOT|~|¬)(.*)$", formula)
    if match_negation:
        inner = parse_formula(match_negation.group(2))  # Parse the inner part after negation
        return Negation(inner)

    # Match implications (e.g., p → q, p IMPLIES q, p => q)
    match_implication = re.match(r"^([a-zA-Z]+)(IMPLIES|→|=>)(.*)$", formula)
    if match_implication:
        premise = Literal(match_implication.group(1))
        conclusion = parse_formula(match_implication.group(3))  # Recursively parse the conclusion
        return Implication(premise, conclusion)

    # Match disjunctions (e.g., p ∨ q, p OR q, p v q)
    match_disjunction = re.match(r"^([a-zA-Z]+)(OR|∨|v)(.*)$", formula)
    if match_disjunction:
        left = Literal(match_disjunction.group(1))
        right = parse_formula(match_disjunction.group(3))  # Recursively parse the right side
        return Disjunction(left, right)

    # Add other parsing logic here for more complex formulas (e.g., conjunctions)

    raise ValueError(f"Formula '{formula}' is not valid.")


# Example usage
if __name__ == '__main__':
    # Create a knowledge base
    kb = KnowledgeBase()

    # Input the knowledge base (KB)
    kb_input = input("Enter your knowledge base (KB) as a list of logical formulas (e.g., 'p→q', 'q→r', '¬p'): ")
    kb_statements = kb_input.split(",")

    # Parse the KB formulas and add them to the knowledge base
    for stmt in kb_statements:
        try:
            kb.add(parse_formula(stmt.strip()))  # Parse and add each formula
        except ValueError as e:
            print(e)

    # Display the knowledge base
    print("\nKnowledge Base:")
    kb.display()

    # Input the alpha (the statement to check)
    alpha_input = input("\nEnter the statement (alpha) to check (e.g., 'q'): ")

    # Parse alpha with error handling
    try:
        alpha = parse_formula(alpha_input.strip())
    except ValueError as e:
        print(f"Error parsing alpha: {e}")
        exit(1)

    # Check if alpha can be inferred from the knowledge base
    can_infer = kb.check_alpha(alpha)
    if can_infer:
        print(f"\nAlpha '{alpha}' can be inferred from the knowledge base.")
    else:
        print(f"\nAlpha '{alpha}' cannot be inferred from the knowledge base.")

Enter your knowledge base (KB) as a list of logical formulas (e.g., 'p→q', 'q→r', '¬p'): q IMPLIES p,p IMPLIES NOT q,q OR  r 

Knowledge Base:
qIMPLIESp
pIMPLIESNOTq
qORr

Enter the statement (alpha) to check (e.g., 'q'): r IMPLIES p

Alpha 'rIMPLIESp' cannot be inferred from the knowledge base.
